# Facebook metrics Data Set - Predicting Number of Comments of Facebook Posts

**To be completed INDIVIDUALLY and due on April 14 at 11:59pm.**

In this assignment, we will work on Facebook posts. The goal is to determine the number of comments a post will receive. Being able to predict the number of comments of a post can be viewed as predicting the impact a post will have, which makes this task quite challenging and interesting. You can find the dataset and details about it [here](https://archive.ics.uci.edu/ml/datasets/Facebook+metrics#) and [here](https://ac.els-cdn.com/S0148296316000813/1-s2.0-S0148296316000813-main.pdf?_tid=1944e5a8-e211-4dbc-bdd6-b233e4df6465&acdnat=1522453539_a59ca4034d01f4654986e45e18e79883). 

The data are stored in a .csv file and each row of the dataset corresponds to a distinct post (data instance). You will evaluate the performance of linear regression on the task of predicting the number of comments of a post.

Relevant Papers/citations: _Prédictions d’activité dans les réseaux sociaux en ligne_ (F. Kawala, A. Douzal-Chouakria, E. Gaussier, E. Dimert), In Actes de la Conférence sur les Modèles et l′Analyse des Réseaux : Approches Mathématiques et Informatique (MARAMI), pp. 16, 2013.

### Task 1

As learned in class most algorithms can only handle numeric values. However, in the given dataset you can see that the values of column 'Type' are categorical. Your first task is to prepare your dataset for linear regression and transform this column into numerical values. Note that creating a mapping that replaces the type field with a single number (e.g. Photo -> 1, Status ->2, Link ->3,...), will not be helpful here since the types are not ordered. Instead, use the "indicator vector" representation discussed in class. 
**(2 pt)**

## Name: Xintong Hao
## GitHub usrname: XintongHao
## Email: hxtong@bu.edu

In [1]:
import pandas as pd
import numpy as np
import csv
%matplotlib inline
%config InlineBackend.figure_format='retina'
# import libraries
import math
import numpy as np
import matplotlib as mp
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
from importlib import reload
from datetime import datetime
from IPython.display import Image
from IPython.display import display_html
from IPython.display import display
from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML

import scipy as sp
import scipy.sparse.linalg as linalg
import scipy.cluster.hierarchy as hr
from scipy.spatial.distance import pdist, squareform

import sklearn.datasets as datasets
import sklearn.metrics as metrics
import sklearn.utils as utils
import sklearn.linear_model as linear_model
import sklearn.model_selection as model_selection
import sklearn.cluster as cluster
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler

import statsmodels.api as sm

from patsy import dmatrices

import seaborn as sns

print('')

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [286]:
df = pd.read_csv('dataset_Facebook.csv')

In [287]:
df = df.drop("Page total likes;Type;Category;Post Month;Post Weekday;Post Hour;Paid;Lifetime Post Total Reach;Lifetime Post Total Impressions;Lifetime Engaged Users;Lifetime Post Consumers;Lifetime Post Consumptions;Lifetime Post Impressions by people who have liked your Page;Lifetime Post reach by people who like your Page;Lifetime People who have liked your Page and engaged with your post;comment;like;share;Total Interactions", axis=1)

In [288]:
csvfile = open( 'dataset_Facebook.csv','r')  

idx = 0
reader = [each for each in csv.DictReader(csvfile, delimiter=';')]
for row in reader:
    df.loc[idx,'Page total likes'] = int(row['Page total likes'])
    df.loc[idx,'Type'] = row['Type']
    df.loc[idx,'Category'] = row['Category']
    df.loc[idx,'Post Month'] = row['Post Month']
    df.loc[idx,'Post Weekday'] = row['Post Weekday']
    df.loc[idx,'Post Hour'] = row['Post Hour']
    df.loc[idx,'Paid'] = row['Paid']
    df.loc[idx,'Lifetime Post Total Reach'] = row['Lifetime Post Total Reach']
    df.loc[idx,'Lifetime Post Total Impressions'] = row['Lifetime Post Total Impressions']
    df.loc[idx,'Lifetime Engaged Users'] = row['Lifetime Engaged Users']
    df.loc[idx,'Lifetime Post Consumers'] = row['Lifetime Post Consumers']
    df.loc[idx,'Lifetime Post Consumptions'] = row['Lifetime Post Consumptions']
    df.loc[idx,'Lifetime Post Impressions by people who have liked your Page'] = row['Lifetime Post Impressions by people who have liked your Page']
    df.loc[idx,'Lifetime Post reach by people who like your Page'] = row['Lifetime Post reach by people who like your Page']
    df.loc[idx,'Lifetime People who have liked your Page and engaged with your post'] = row['Lifetime People who have liked your Page and engaged with your post']
    df.loc[idx,'comment'] = row['comment']
    df.loc[idx,'like'] = row['like']
    df.loc[idx,'share'] = row['share']
    idx += 1

In [289]:
pd.set_option("display.max_rows",600)
pd.get_option("display.max_rows")

600

In [290]:
df['Page total likes'] = df['Page total likes'].astype('int64')
df['Category'] = df['Category'].astype('int64')
df['Post Month'] = df['Post Month'].astype('int64')
df['Post Weekday'] = df['Post Weekday'].astype('int64')
df['Post Hour'] = df['Post Hour'].astype('int64')
# df['Paid'] = df['Paid'].astype('int64')
df['Lifetime Post Total Reach'] = df['Lifetime Post Total Reach'].astype('int64')
df['Lifetime Post Total Impressions'] = df['Lifetime Post Total Impressions'].astype('int64')
df['Lifetime Engaged Users'] = df['Lifetime Engaged Users'].astype('int64')
df['Lifetime Post Consumers'] = df['Lifetime Post Consumers'].astype('int64')
df['Lifetime Post Consumptions'] = df['Lifetime Post Consumptions'].astype('int64')
df['Lifetime Post Impressions by people who have liked your Page'] = df['Lifetime Post Impressions by people who have liked your Page'].astype('int64')
df['Lifetime Post reach by people who like your Page'] = df['Lifetime Post reach by people who like your Page'].astype('int64')
df['Lifetime People who have liked your Page and engaged with your post'] = df['Lifetime People who have liked your Page and engaged with your post'].astype('int64')
df['comment'] = df['comment'].astype('int64')
# df['like'] = df['like'].astype('int64')
# df['share'] = df['share'].astype('int64')

In [291]:
for idx, row in df.iterrows():
    if row['Paid'] == '':
        df.loc[idx,'Paid'] = 0
    if row['like'] == '':
        df.loc[idx,'like'] = 0
    if row['share'] == '':
        df.loc[idx,'share'] = 0
df['Paid'] = df['Paid'].astype('int64')
df['like'] = df['like'].astype('int64')
df['share'] = df['share'].astype('int64')

In [292]:
df = pd.get_dummies(df, columns=['Type'], prefix='Type')
df.head()

,Page total likes,Category,Post Month,Post Weekday,Post Hour,Paid,Lifetime Post Total Reach,Lifetime Post Total Impressions,Lifetime Engaged Users,Lifetime Post Consumers,...,Lifetime Post Impressions by people who have liked your Page,Lifetime Post reach by people who like your Page,Lifetime People who have liked your Page and engaged with your post,comment,like,share,Type_Link,Type_Photo,Type_Status,Type_Video
0,139441,2,12,4,3,0,2752,5091,178,109,...,3078,1640,119,4,79,17,0,1,0,0
1,139441,2,12,3,10,0,10460,19057,1457,1361,...,11710,6112,1108,5,130,29,0,0,1,0
2,139441,3,12,3,3,0,2413,4373,177,113,...,2812,1503,132,0,66,14,0,1,0,0
3,139441,2,12,2,10,1,50128,87991,2211,790,...,61027,32048,1386,58,1572,147,0,1,0,0
4,139441,2,12,2,3,0,7244,13594,671,410,...,6228,3200,396,19,325,49,0,1,0,0


In [293]:
df['Type_Link'] = df['Type_Link'].astype('int64')
df['Type_Photo'] = df['Type_Photo'].astype('int64')
df['Type_Status'] = df['Type_Status'].astype('int64')
df['Type_Video'] = df['Type_Video'].astype('int64')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 21 columns):
Page total likes                                                       500 non-null int64
Category                                                               500 non-null int64
Post Month                                                             500 non-null int64
Post Weekday                                                           500 non-null int64
Post Hour                                                              500 non-null int64
Paid                                                                   500 non-null int64
Lifetime Post Total Reach                                              500 non-null int64
Lifetime Post Total Impressions                                        500 non-null int64
Lifetime Engaged Users                                                 500 non-null int64
Lifetime Post Consumers                                                500 non-null i

### Task 2

Now your dataset is ready for linear regression. To familiarize yourself with this task, here simply use the following [package](http://www.statsmodels.org/dev/regression.html) to perform regression on the dataset. The dependent variable should be the column 'comment' and the remaining columns of your dataset should be the independent variables. Split your dataset into 80% and 20% for training and testing, respectively. Report the linear model's summary as shown [here](https://www.statsmodels.org/stable/index.html) by using the .summary() function after fitting your model and compute the error of your prediction using the [mean squared error (MSE)](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.mean_squared_error.html).  **(2 pt)**

In [294]:
new_column_names = [x.replace(' ','_') for x in df.columns]
df.columns = new_column_names

In [295]:
X = df[df.columns.difference(['comment'])]
# comment = df.pop('comment')
y = df["comment"]

In [301]:
train_set_size = int(0.8*len(df.index))
train_set_size

400

In [302]:
X_train = X[:train_set_size]  # selects first 400 rows (examples) for train set
y_train = y[:train_set_size]
X_test = X[train_set_size:]   # selects from row 100 until the last one for test set
y_test = y[train_set_size:]
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(400, 20) (400,)
(100, 20) (100,)


In [303]:
model = sm.OLS(y_train, X_train)
results = model.fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                comment   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.797
Method:                 Least Squares   F-statistic:                     83.68
Date:                Tue, 10 Apr 2018   Prob (F-statistic):          6.44e-123
Time:                        21:25:47   Log-Likelihood:                -1480.3
No. Observations:                 400   AIC:                             3001.
Df Residuals:                     380   BIC:                             3080.
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                                                                          coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

In [304]:
y_pred = results.predict(X_test)

In [305]:
from sklearn.metrics import mean_squared_error

In [306]:
mean_squared_error(y_test, y_pred)

129.94574745864963

Provide an explanation of the OLS regression results. In particular, briefly explain and interpret: 

- R-squared, 

- coeff, 

Finally, 

- which are the statistically significant predictors and why? 

**(1 pt)** 

### R-squared = 0.807,  Adj. R-squared = 0.797
R-squared is a statistical measure of how well the regression line approximates the real data points.
In my model, R-squared = 0.807, it means 80.7% of variation in comments is explained by our independent variables
The Adj.R-squared is a modified version of R-squared that has been adjusted for the number of predictors in the model. It increases only if the new term improves the model more than would be expected by chance. It decreases when a predictor improves the model by less than expected by chance. As the value of Adj. R-squared in my model, it is always lower than the R-squared.
However, R-squared cannot determine whether the coefficient estimates and predictions are biased, which is why you must assess the residual plots. My model is still likely that has high R-squared value for a model that does not fit the data.


Reference: http://blog.minitab.com/blog/adventures-in-statistics-2/regression-analysis-how-do-i-interpret-r-squared-and-assess-the-goodness-of-fit

## confidence interval and coeff of each independent variables are shown below 

In [169]:
print('Confidence Intervals: \n {}'.format(results.conf_int()))
print('')
print('Parameters: {}'.format(results.params))

Confidence Intervals: 
                                                             0          1
Page total likes                                    -0.000084   0.000570
Category                                            -4.158842  -1.373047
Post Month                                          -1.815467   0.481490
Post Weekday                                        -0.701234   0.289240
Post Hour                                           -0.042954   0.452030
Paid                                                -2.267065   2.239045
Lifetime Post Total Reach                           -0.000527   0.000114
Lifetime Post Total Impressions                     -0.000077   0.000232
Lifetime Engaged Users                              -0.056658   0.011561
Lifetime Post Consumers                             -0.000157   0.062821
Lifetime Post Consumptions                          -0.000278   0.001245
Lifetime Post Impressions by people who have li...  -0.000392   0.000279
Lifetime Post reach by peop

## coeff
The positive coeff parameter estimate implies that the independent variable has a positive effect on comment, otherwise, it has a negative effect on comment.


## confidence interval
If the idependent variable's confidence interval for the parameter includes zero, it may not have any predivtive value, which means it is not statistically significant. Meanwhile, the p-value less than 0.05 for the parameter implies that the effect of the independent variable is statistically significant (using p < 0.05 as a rejection rule).

In [185]:
independ_var = ['Page total likes', 'Category', 'Post Month', 'Post Weekday',
       'Post Hour', 'Paid', 'Lifetime Post Total Reach',
       'Lifetime Post Total Impressions', 'Lifetime Engaged Users',
       'Lifetime Post Consumers', 'Lifetime Post Consumptions',
       'Lifetime Post Impressions by people who have liked your Page',
       'Lifetime Post reach by people who like your Page',
       'Lifetime People who have liked your Page and engaged with your post',
       'like', 'share', 'Type_Link', 'Type_Photo', 'Type_Status',
       'Type_Video']
model = sm.OLS(y_train.values, X_train.values)
results = model.fit()
CIs = results.conf_int()
notSignificant = (CIs[:,0] < 0) & (CIs[:,1] > 0)
j = 0
print('If the independant variable is statistically significant:')
for i in notSignificant:
    print(independ_var[j] + ' : ' + str(i))
    j += 1

If the independant variable is statistically significant:
Page total likes : True
Category : False
Post Month : True
Post Weekday : True
Post Hour : True
Paid : True
Lifetime Post Total Reach : True
Lifetime Post Total Impressions : True
Lifetime Engaged Users : True
Lifetime Post Consumers : True
Lifetime Post Consumptions : True
Lifetime Post Impressions by people who have liked your Page : True
Lifetime Post reach by people who like your Page : True
Lifetime People who have liked your Page and engaged with your post : False
like : False
share : False
Type_Link : True
Type_Photo : True
Type_Status : True
Type_Video : True


###  From the results above, we can see that the statistically significant predictors are:
### [ "Category", "Lifetime People who have liked your Page and engaged with your post", "like", "share"]

### Task 3

We will attempt to improve the MSE. In order to do so we are going to use regularization by returning a regularized fit to the linear regression model. Note that regularization's goal is to reduce overfitting, that is it creates a more generalized linear model and removes possible noise from the dataset. You will complete this task by using the .fit_regularized() method, and more specifically the **ridge fit**. More details can be found [here](http://www.statsmodels.org/dev/generated/statsmodels.regression.linear_model.OLS.fit_regularized.html). Perform Kfold cross-validation, where K = 5, to both the linear model and the regularized linear model, in order to decide which performs better. Report the MSE of the two models. Remember to shuffle the data before performing cross-validation. **(2 pt)**

In [374]:
MSE_naive = []
MSE_reg = []
for k in range(5):
    X, y = utils.shuffle(X, y, random_state=1)
    # Split into training and test sets
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)
    
    model_naive = sm.OLS(y_train, X_train)
    results_naive = model_naive.fit()
    y_pred_naive = results_naive.predict(X_test)
    MSE_naive.append(mean_squared_error(y_test, y_pred_naive))
    
    results_reg = sm.OLS(y_train, X_train).fit_regularized(alpha = 5, L1_wt=0)
    y_pred_reg = results_reg.predict(X_test)
    MSE_reg.append(mean_squared_error(y_test, y_pred_reg))
    

In [375]:
print("MSE of linear model is {}".format(np.mean(MSE_naive)))
print("MSE of regularized linear model is {}".format(np.mean(MSE_reg)))

MSE of linear model is 117.69009601232752
MSE of regularized linear model is 108.06449558381075


### From the MSE value above, we can decide that the regularized linear model performs better when alpha=5

Try the following values for the ridge regression parameter $\alpha$ (the penalty weight), $\alpha = [0,5,10,100]$.**(1 pt)**

In [336]:
alphas_to_try = np.array([0,5,10,100])
alphas_to_try

array([  0,   5,  10, 100])

In [339]:
X, y = utils.shuffle(X, y, random_state=1)
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)
for this_alpha in alphas_to_try:
    model_reg = sm.OLS(y_train, X_train).fit_regularized(method='elastic_net', alpha=this_alpha,L1_wt=0)
    y_predicted = model_reg.predict(X_test)
    MSE = mean_squared_error(y_test, y_predicted)
#     print(model_reg.params)
    print("alpha = {}, MSE = {}".format(this_alpha, MSE))


alpha = 0, MSE = 224.50761968350866
alpha = 5, MSE = 190.11504284467802
alpha = 10, MSE = 191.68947318576738
alpha = 100, MSE = 193.47187206446935


Based on your knowledge and your results, as a data scientist would you prefer a linear model, or a regularized linear model? What effects can a zero (or very small) $\alpha$ and a very large $\alpha$ cause?**(1 pt)** 

#### When there are many features in the data, I prefer regularized linear model, because it has better performance when dealing with overfitting problem.

#### α is a zero (or very samll):
The regularization term will become nearly zero. As a result, the cost function mostly depends on the MSE term just like before applying regularization. Or we can say that, when α is nearly zero, the regularization term won’t have any significant effect on shrinking the parameters. Therefore the model is more likely to overfit.

#### α is very large:
α is very large means we put much weight in the regularization term. And as a result, the parameters will shrink to a very small values. That approach, however, brings a real problem, that is, rather than preventing Overfitting, our parameters now become so small so that it can’t even fit the training data well, or we can say that: applying so much regularization cause our model to underfit the dataset.

Reference :https://chunml.github.io/ChunML.github.io/tutorial/Regularization/

### Task 4

Another way that may help us improve the MSE is by adding quadratic terms of significant variables in the model. Add 2 quadratic terms in your **regularized linear model** based on two significant variables. An interesting and intuitive tutorial about understanding which are the significant variables and the importance of quadratic terms can be found [here](http://statisticsbyjim.com/regression/interpret-coefficients-p-values-regression/). Note that in order to add the quadratic terms you will need to change the formula of your model, as in [example](http://www.statsmodels.org/dev/examples/notebooks/generated/formulas.html) and in [example](https://stackoverflow.com/questions/31978948/python-stats-models-quadratic-term-in-regression/36539157). Perform Kfold cross-validation, where K = 5, to both the linear model and the regularized linear model with quadratic terms, in order to decide which performs better. Report the MSE of the two models. Remember to shuffle the data before performing cross-validation. **(2 pt)**

## Two significant variables: Category and like

In [204]:
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols, glm

In [376]:
my_formula = 'comment ~ Page_total_likes + Category + Post_Month + Post_Weekday + Post_Hour \
+ Paid + Lifetime_Post_Total_Reach + Lifetime_Post_Total_Impressions + Lifetime_Engaged_Users \
+ Lifetime_Post_Consumers + Lifetime_Post_Consumptions + Lifetime_Post_Impressions_by_people_who_have_liked_your_Page\
+ Lifetime_Post_reach_by_people_who_like_your_Page + Lifetime_People_who_have_liked_your_Page_and_engaged_with_your_post \
+ like + share + Type_Link + Type_Photo + Type_Status + Type_Video\
+ np.power(Category, 2) + np.power(like, 2)'

In [377]:
MSE_qua_naive = []
MSE_qua_reg = []

for k in range(5):
    X, y = utils.shuffle(X, y, random_state=1)
    # Split into training and test sets
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)
    df_train = pd.concat([X_train, y_train], axis=1)
    
    results_qua_naive = ols(formula = my_formula, data=df_train  ).fit()
    y_pred_qua_naive = results_qua_naive.predict(X_test)
    MSE_qua_naive.append(mean_squared_error(y_test, y_pred_qua_naive))
    
    results_qua_reg = ols(formula = my_formula, data=df_train).fit_regularized(alpha = 5, L1_wt=0)
    y_pred_qua_reg = results_qua_reg.predict(X_test)
    MSE_qua_reg.append(mean_squared_error(y_test, y_pred_qua_reg))

In [378]:
print("MSE of linear model is {}".format(np.mean(MSE_qua_naive)))
print("MSE of regularized linear model is {}".format(np.mean(MSE_qua_reg)))

MSE of linear model is 63.433622949801986
MSE of regularized linear model is 60.7821451124477


What issues do you think could come up if we used too many quadratic or cubic terms in our model?**(1 pt)** 

#### 1. It will cause serious multicollinearity
#### 2. When the size of predictor is large, it may cause numerical overflow
#### 3. If we extrapolate beyond the limits of your observed values, it may produce meaningless results beyongd the scope of the model.
